In [1]:
from google.colab import drive as MountDrive
MountDrive.mount('/content/drive',force_remount=True)

Mounted at /content/drive


In [2]:
pip install gspread

In [3]:
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())


In [4]:
import os
os.chdir("/content/drive/MyDrive/CS 245 Project/CS245 Data")

In [5]:
ls -l

total 36437
-rw------- 1 root root   995635 Nov 30 03:26 covid_data_hospitalization.csv
-rw------- 1 root root      151 Dec  5 20:51 covid_data_hospitalization.gsheet
-rw------- 1 root root  3441829 Dec  6 02:43 covid_data_vaccines2.xlsx
-rw------- 1 root root      151 Dec  6 03:47 covid_data_VaccinesAndMask.gsheet
-rw------- 1 root root  2126695 Dec  6 02:49 covid_data_VaccinesAndMask.xlsx
-rw------- 1 root root  2075453 Nov 30 03:24 covid_data_vaccines.xlsx
-rw------- 1 root root  1341657 Nov 28 21:25 covid_data.xlsx
-rw------- 1 root root   343667 Nov 28 21:32 Data_Extraction.ipynb
-rw------- 1 root root   159488 Nov 28 21:31 state_temp.xlsx
-rw------- 1 root root    86928 Nov 30 03:28 test_covid_hospitalization.npy
-rw------- 1 root root    86928 Nov 30 03:27 test_covid_vaccines.npy
-rw------- 1 root root  2976128 Nov 28 21:25 test_cs245.npy
-rw------- 1 root root   652528 Nov 30 03:27 train_covid_vaccine.npy
-rw------- 1 root root 22368128 Nov 28 21:25 train_cs245.npy
-rw------- 1

In [6]:
#worksheet = gc.open('covid_data').sheet1

###PUJA####
worksheet = gc.open('covid_data_VaccinesAndMask').sheet1

######
# get_all_values gives a list of rows.
rows = worksheet.get_all_values()
col = rows[0]
print(rows)

# Convert to a DataFrame and render.
import pandas as pd
pd.DataFrame.from_records(rows)


[['Date', 'state', 'new_case', 'new_death', 'tot_cases', 'tot_death', 'Administered_Dose1_Recip', 'Series_Complete_Yes', 'mask_mandate'], ['1/12/2021', 'Alabama', '3.86E+03', '1.04E+02', '4.09E+05', '9.05E+03', '0.00E+00', '0.00E+00', '1.00E+00'], ['1/13/2021', 'Alabama', '3.85E+03', '1.27E+02', '4.13E+05', '9.17E+03', '0.00E+00', '0.00E+00', '1.00E+00'], ['1/14/2021', 'Alabama', '3.22E+03', '7.70E+01', '4.16E+05', '9.25E+03', '0.00E+00', '0.00E+00', '1.00E+00'], ['1/15/2021', 'Alabama', '3.19E+03', '8.00E+01', '4.19E+05', '9.33E+03', '0.00E+00', '0.00E+00', '1.00E+00'], ['1/16/2021', 'Alabama', '1.91E+03', '3.90E+01', '4.21E+05', '9.37E+03', '0.00E+00', '0.00E+00', '1.00E+00'], ['1/17/2021', 'Alabama', '1.49E+03', '3.50E+01', '4.22E+05', '9.40E+03', '0.00E+00', '0.00E+00', '1.00E+00'], ['1/18/2021', 'Alabama', '2.44E+03', '6.50E+01', '4.25E+05', '9.47E+03', '0.00E+00', '0.00E+00', '1.00E+00'], ['1/19/2021', 'Alabama', '3.28E+03', '8.00E+01', '4.28E+05', '9.55E+03', '0.00E+00', '0.00E+

,0,1,2,3,4,5,6,7,8
0,Date,state,new_case,new_death,tot_cases,tot_death,Administered_Dose1_Recip,Series_Complete_Yes,mask_mandate
1,1/12/2021,Alabama,3.86E+03,1.04E+02,4.09E+05,9.05E+03,0.00E+00,0.00E+00,1.00E+00
2,1/13/2021,Alabama,3.85E+03,1.27E+02,4.13E+05,9.17E+03,0.00E+00,0.00E+00,1.00E+00
3,1/14/2021,Alabama,3.22E+03,7.70E+01,4.16E+05,9.25E+03,0.00E+00,0.00E+00,1.00E+00
4,1/15/2021,Alabama,3.19E+03,8.00E+01,4.19E+05,9.33E+03,0.00E+00,0.00E+00,1.00E+00
...,...,...,...,...,...,...,...,...,...
15646,11/16/2021,Wyoming,3.10E+02,4.90E+01,1.08E+05,1.35E+03,3.04E+05,2.60E+05,0.00E+00
15647,11/17/2021,Wyoming,2.45E+02,0.00E+00,1.09E+05,1.35E+03,3.04E+05,2.60E+05,0.00E+00
15648,11/18/2021,Wyoming,4.25E+02,0.00E+00,1.09E+05,1.35E+03,3.05E+05,2.61E+05,0.00E+00
15649,11/19/2021,Wyoming,2.35E+02,0.00E+00,1.09E+05,1.35E+03,3.05E+05,2.61E+05,0.00E+00


In [7]:
import numpy as np

In [8]:
data = np.array(rows)
data.shape

(15651, 9)

In [9]:
#removing column types
columns = data[0,:]
data = data[1:,:]
columns

array(['Date', 'state', 'new_case', 'new_death', 'tot_cases', 'tot_death',
       'Administered_Dose1_Recip', 'Series_Complete_Yes', 'mask_mandate'],
      dtype='<U24')

In [10]:
#removing dates and state names
date_names = data[:,0:2]
data = data[:,2:]
date_names

array([['1/12/2021', 'Alabama'],
       ['1/13/2021', 'Alabama'],
       ['1/14/2021', 'Alabama'],
       ...,
       ['11/18/2021', 'Wyoming'],
       ['11/19/2021', 'Wyoming'],
       ['11/20/2021', 'Wyoming']], dtype='<U24')

In [11]:
states = 50
days = int(data.shape[0]/states) #for 50 states
days

313

In [12]:
# change where deaths column is, since code expect it at column 1 (zero based)
data[:,1],data[:,2] = data[:,2], data[:,1]

In [13]:
#reshape data to following: (days,states,features)
data = data.reshape(days,states,-1)

#rotate axis to have : (states,days,features)
data = np.swapaxes(data,0,1)
data.shape

(50, 313, 7)

In [15]:
#get temperature data
worksheet = gc.open('state_temp').sheet1

# get_all_values gives a list of rows.
temperature = worksheet.get_all_values()
col = temperature[0]
print(temperature)

# Convert to a DataFrame and render.
import pandas as pd
pd.DataFrame.from_records(rows)


[['Date', '2021-01-12', '2021-01-13', '2021-01-14', '2021-01-15', '2021-01-16', '2021-01-17', '2021-01-18', '2021-01-19', '2021-01-20', '2021-01-21', '2021-01-22', '2021-01-23', '2021-01-24', '2021-01-25', '2021-01-26', '2021-01-27', '2021-01-28', '2021-01-29', '2021-01-30', '2021-01-31', '2021-02-01', '2021-02-02', '2021-02-03', '2021-02-04', '2021-02-05', '2021-02-06', '2021-02-07', '2021-02-08', '2021-02-09', '2021-02-10', '2021-02-11', '2021-02-12', '2021-02-13', '2021-02-14', '2021-02-15', '2021-02-16', '2021-02-17', '2021-02-18', '2021-02-19', '2021-02-20', '2021-02-21', '2021-02-22', '2021-02-23', '2021-02-24', '2021-02-25', '2021-02-26', '2021-02-27', '2021-02-28', '2021-03-01', '2021-03-02', '2021-03-03', '2021-03-04', '2021-03-05', '2021-03-06', '2021-03-07', '2021-03-08', '2021-03-09', '2021-03-10', '2021-03-11', '2021-03-12', '2021-03-13', '2021-03-14', '2021-03-15', '2021-03-16', '2021-03-17', '2021-03-18', '2021-03-19', '2021-03-20', '2021-03-21', '2021-03-22', '2021-03-2

,0,1,2,3,4,5,6,7,8
0,Date,state,new_case,new_death,tot_cases,tot_death,Administered_Dose1_Recip,Series_Complete_Yes,mask_mandate
1,1/12/2021,Alabama,3.86E+03,1.04E+02,4.09E+05,9.05E+03,0.00E+00,0.00E+00,1.00E+00
2,1/13/2021,Alabama,3.85E+03,1.27E+02,4.13E+05,9.17E+03,0.00E+00,0.00E+00,1.00E+00
3,1/14/2021,Alabama,3.22E+03,7.70E+01,4.16E+05,9.25E+03,0.00E+00,0.00E+00,1.00E+00
4,1/15/2021,Alabama,3.19E+03,8.00E+01,4.19E+05,9.33E+03,0.00E+00,0.00E+00,1.00E+00
...,...,...,...,...,...,...,...,...,...
15646,11/16/2021,Wyoming,3.10E+02,4.90E+01,1.08E+05,1.35E+03,3.04E+05,2.60E+05,0.00E+00
15647,11/17/2021,Wyoming,2.45E+02,0.00E+00,1.09E+05,1.35E+03,3.04E+05,2.60E+05,0.00E+00
15648,11/18/2021,Wyoming,4.25E+02,0.00E+00,1.09E+05,1.35E+03,3.05E+05,2.61E+05,0.00E+00
15649,11/19/2021,Wyoming,2.35E+02,0.00E+00,1.09E+05,1.35E+03,3.05E+05,2.61E+05,0.00E+00


In [16]:
temp = np.array(temperature)
temp.shape

(51, 314)

In [17]:
# remove column types
col_types = temp[0,:]
temp = temp[1:,:]
print(col_types)

['Date' '2021-01-12' '2021-01-13' '2021-01-14' '2021-01-15' '2021-01-16'
 '2021-01-17' '2021-01-18' '2021-01-19' '2021-01-20' '2021-01-21'
 '2021-01-22' '2021-01-23' '2021-01-24' '2021-01-25' '2021-01-26'
 '2021-01-27' '2021-01-28' '2021-01-29' '2021-01-30' '2021-01-31'
 '2021-02-01' '2021-02-02' '2021-02-03' '2021-02-04' '2021-02-05'
 '2021-02-06' '2021-02-07' '2021-02-08' '2021-02-09' '2021-02-10'
 '2021-02-11' '2021-02-12' '2021-02-13' '2021-02-14' '2021-02-15'
 '2021-02-16' '2021-02-17' '2021-02-18' '2021-02-19' '2021-02-20'
 '2021-02-21' '2021-02-22' '2021-02-23' '2021-02-24' '2021-02-25'
 '2021-02-26' '2021-02-27' '2021-02-28' '2021-03-01' '2021-03-02'
 '2021-03-03' '2021-03-04' '2021-03-05' '2021-03-06' '2021-03-07'
 '2021-03-08' '2021-03-09' '2021-03-10' '2021-03-11' '2021-03-12'
 '2021-03-13' '2021-03-14' '2021-03-15' '2021-03-16' '2021-03-17'
 '2021-03-18' '2021-03-19' '2021-03-20' '2021-03-21' '2021-03-22'
 '2021-03-23' '2021-03-24' '2021-03-25' '2021-03-26' '2021-03-27'
 '2

In [18]:
#remove state names
states = temp[:,0]
temp = temp[:,1:]

In [19]:
temp = np.expand_dims(temp, axis=2)
data = np.append(data,temp,axis=2)

In [20]:
#temperature is now the last column
data = data.astype('float64')
data.shape

(50, 313, 8)

In [24]:
pwd


'/content/drive/.shortcut-targets-by-id/1nMf4IVSJE73nIq84zw9j3o3qGsyQeGCv/CS 245 Project/CS245 Data'

# Save our data to google drive

In [26]:
training = data[:,0:233,:]
#np.save('/content/drive/MyDrive/CG-ODE-main/CG-ODE-main/data/Dec/train_cs245.npy',training) #change location as needed

#### PUJA 
np.save('train_VacMask_cs245.npy',training) #change location as needed
#####

In [27]:
testing = data[:,233:233+31,:]
#np.save('/content/drive/MyDrive/CG-ODE-main/CG-ODE-main/data/Dec/test_cs245.npy',testing)

#### PUJA
np.save('test_VacMask_cs245.npy',testing)
####